# Magics

In [1]:
%load_ext autoreload

%autoreload 2

# Imports

In [2]:
import os
import gc
import configparser
import pathlib as p
import numpy as np
import seaborn as sns

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt 

In [4]:
import pandas as pd
pd.options.display.max_columns = 999

In [5]:
import torch

In [6]:
import clipper_python as clipper

In [7]:
import torch.nn as nn
import torch.optim as optim

In [8]:
from frag_nn.pytorch.network import GNINA_regressor, GNINA_regressor_v2, GNINA_regressor_v3, GNINA_regressor_v4, GNINA_regressor_v5, GNINA_regressor_v6, GNINA_regressor_v7, GNINA_regressor_v8, GNINA_regressor_v9, GNINA_regressor_v10
from frag_nn.pytorch.network import ClassifierV1
from frag_nn.data import XChemData
from frag_nn.pytorch.dataset import EventDataset
import frag_nn.constants as c


# Get Config

In [9]:
config_path = "/dls/science/groups/i04-1/conor_dev/pandda_nn/frag_nn/params.ini"

In [10]:
conf = configparser.ConfigParser()

In [11]:
conf.read(config_path)

['/dls/science/groups/i04-1/conor_dev/pandda_nn/frag_nn/params.ini']

In [12]:
ds_conf = conf[c.x_chem_database]

In [13]:
grid_size = 48
grid_step = 0.5
network_type = "classifier"
network_version = 10
dataset_version = 2

# Load Database

In [14]:
x_chem_dataset = XChemData(host=ds_conf[c.db_host], 
                         port=ds_conf[c.db_port], 
                         database=ds_conf[c.db_database], 
                         user=ds_conf[c.db_user], 
                         password=ds_conf[c.db_password])


In [15]:
events = x_chem_dataset.get_database("pandda_event")

In [16]:
events

,id,event,event_centroid_x,event_centroid_y,event_centroid_z,event_dist_from_site_centroid,lig_centroid_x,lig_centroid_y,lig_centroid_z,lig_dist_event,lig_id,pandda_event_map_native,pandda_model_pdb,pandda_input_mtz,pandda_input_pdb,crystal_id,pandda_run_id,site_id,pandda_event_map_cut,ligand_confidence_inspect,ligand_confidence,ligand_confidence_source,comment,data_proc_id,refinement_id,interesting,event_status,created_date,modified_date
0,89808,1,-25.714760,-18.774393,74.669490,115.938375513,-26.127133,-19.161533,75.524867,0.426227,LIG E 1,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,724023,848,208499,None,High,4 - High Confidence,SD,no comment,685679,700791,False,4,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.165243+00:00
1,89817,1,-10.306649,-45.824505,56.227310,104.347246519,11.036231,-37.789615,57.381923,3.301904,LIG E 1,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,724109,848,208501,None,Low,Low,SD,no comment,685765,700877,False,4,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.169057+00:00
2,174128,3,11.512784,13.339129,14.711447,36.9053117894,11.926500,15.062125,16.442688,2.753091,LIG D 2,/dls/labxchem/data/2018/lb19400-1/processing/a...,/dls/labxchem/data/2018/lb19400-1/processing/a...,/dls/labxchem/data/2018/lb19400-1/processing/a...,/dls/labxchem/data/2018/lb19400-1/processing/a...,889075,912,290029,None,Low,Low,SD,no comment,849098,864210,False,5,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.191238+00:00
3,88668,1,-4.089268,-24.952103,27.085863,54.8430717837,-5.072333,-26.251600,26.630467,1.956114,LIG E 1,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,723859,357,5915,None,High,1 - Low Confidence,SD,no comment,685515,700627,False,1,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.201871+00:00
4,88658,2,14.962372,-32.484849,72.617364,114.17802315,6.449429,-41.220071,73.028214,2.553417,LIG E 2,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,723882,357,5913,None,High,High,SD,no comment,685538,700650,False,4,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.204472+00:00
5,88662,3,0.553781,-25.862561,57.607982,88.2570693118,-0.239176,-27.414059,58.268706,1.935437,LIG E 1,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,723915,357,5914,None,High,4 - High Confidence,SD,no comment,685571,700683,False,4,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.207134+00:00
6,88656,1,11.279475,-30.903472,84.656118,121.380026328,12.795600,-32.441733,82.583200,6.233013,LIG E 3,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,723898,357,5913,None,High,4 - High Confidence,SD,no comment,685554,700666,False,4,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.209883+00:00
7,88660,2,15.498963,-27.696655,70.277145,111.491708581,-14.639000,-21.202833,49.422611,16.346269,LIG E 1,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,/dls/labxchem/data/2018/lb19990-1/processing/a...,723862,357,5913,None,High,0 - no ligand present,SD,no comment,685518,700630,False,4,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.212

# Get accessible events

In [17]:
accessible_events_list = []

for pth in events["pandda_input_mtz"]:
    try:
        if p.Path(pth).exists():
            accessible_events_list.append(True)
        else:
            accessible_events_list.append(False)
            
    except PermissionError as e:
        accessible_events_list.append(False)


In [18]:
accessible_events_mask = np.array(accessible_events_list)

In [19]:
np.unique(accessible_events_mask,
               return_counts=True)

(array([False,  True]), array([4523, 1869]))

In [20]:
accessible_events = events[accessible_events_mask]

In [21]:
accessible_events

,id,event,event_centroid_x,event_centroid_y,event_centroid_z,event_dist_from_site_centroid,lig_centroid_x,lig_centroid_y,lig_centroid_z,lig_dist_event,lig_id,pandda_event_map_native,pandda_model_pdb,pandda_input_mtz,pandda_input_pdb,crystal_id,pandda_run_id,site_id,pandda_event_map_cut,ligand_confidence_inspect,ligand_confidence,ligand_confidence_source,comment,data_proc_id,refinement_id,interesting,event_status,created_date,modified_date
10,10462,3,-29.248508,-26.263103,36.117107,75.4091519705,-14.993692,-6.399077,40.609000,6.383642,LIG D 1,/dls/labxchem/data/2017/lb18145-21/processing/...,/dls/labxchem/data/2017/lb18145-21/processing/...,/dls/labxchem/data/2017/lb18145-21/processing/...,/dls/labxchem/data/2017/lb18145-21/processing/...,502271,753,12516,None,None,None,NA,no comment,481995,488760,False,5,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:06.231426+00:00
309,4471,1,-8.831627,6.976942,-10.706931,21.8202736505,-9.614056,7.171889,-11.129722,0.741874,LIG D 1,/dls/labxchem/data/2017/lb18145-31/processing/...,/dls/labxchem/data/2017/lb18145-31/processing/...,/dls/labxchem/data/2017/lb18145-31/processing/...,/dls/labxchem/data/2017/lb18145-31/processing/...,48691,442,7182,None,Low,1 - Low Confidence,SD,no comment,48691,48691,False,3,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:07.619287+00:00
381,221902,1,58.909012,-54.903488,83.510073,161.877869138,58.558000,-55.179267,82.402867,1.592679,LIG E 1,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,1041698,814,13787,None,None,None,NA,no comment,998504,1013616,False,5,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:10.152253+00:00
402,221905,1,58.695595,-52.674826,80.216372,159.374791007,58.582267,-54.545133,82.608400,1.515475,LIG E 1,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,1041608,814,13787,None,None,None,NA,no comment,998414,1013526,False,5,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:10.645265+00:00
419,221907,1,57.295201,-51.230239,81.836399,159.217842209,58.722313,-54.257063,82.736125,1.536252,LIG E 2,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,1042168,814,13787,None,None,None,NA,no comment,998974,1014086,False,5,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:11.200683+00:00
457,221909,1,59.066307,-55.587837,84.260969,162.556435544,58.859800,-55.707667,83.415333,2.381993,LIG E 1,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,/dls/labxchem/data/2017/lb18145-47/processing/...,1041692,814,13787,None,None,None,NA,no comment,998498,1013610,False,5,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:11.805454+00:00
467,6543,1,31.624118,45.732885,24.029803,84.7086764654,31.116933,46.303067,23.223067,0.851603,LIG E 1,/dls/labxchem/data/2017/lb13385-87/processing/...,/dls/labxchem/data/2017/lb13385-87/processing/...,/dls/labxchem/data/2017/lb13385-87/processing/...,/dls/labxchem/data/2017/lb13385-87/processing/...,21550,611,10587,None,None,None,NA,no comment,21550,21550,False,3,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:11.902156+00:00
468,105853,1,12.658549,55.687565,26.775061,87.6314684678,12.666333,55.702467,23.468000,0.641944,LIG B 1,/dls/labxchem/data/2017/lb13385-109/processing...,/dls/labxchem/data/2017/lb13385-109/processing...,/dls/labxchem/data/2017/lb13385-109/processing...,/dls/labxchem/data/2017/lb13385-109/processing...,769244,617,10788,None,None,None,NA,no comment,730089,745201,False,1,2019-03-04 13:19:01.925640+00:00,2019-03-05 14:27:12.010842+00:00
469,105854,1,13.159956,54.043988,2

In [22]:
split = np.random.rand(len(accessible_events)) < 0.9

In [23]:
events_train = accessible_events[split]#
print(len(events_train))

1688


In [24]:
events_test = accessible_events[~split]
print(len(events_test))

181


In [25]:
events_train = pd.read_csv("new_events_train.csv")

/dls/science/groups/i04-1/conor_dev/anaconda/envs/env_pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
events_train

Unnamed: 0  1-BDC Comment  Interesting Ligand Confidence  \
0              23   0.49    None        False               Low   
1              24   0.23    None        False               Low   
2              25   0.22    None        False               Low   
3              26   0.15    None        False               Low   
4              27   0.12    None        False               Low   
5              28   0.23    None        False               Low   
6              29   0.14    None        False               Low   
7              30   0.51    None        False               Low   
8              32   0.08    None        False               Low   
9              33   0.15    None        False               Low   
10             34   0.13    None        False               Low   
11             35   0.30    None        False               Low   
12             37   0.13    None        False               Low   
13             38   0.25    None        False               Low   
14             39   0.37    None        False               Low   
15             40   0.33    None        False               Low   
16             41   0.24    None        False               Low   
17             42   0.22    None        False               Low   
18             43   0.64    None        False               Low   
19             44   0.25    None        False               Low   
20             45   0.40    None        False               Low   
21             47   0.06    None        False               Low   
22             48   0.15    None        False               Low   
23             49   0.12    None        False               Low   
24             50   0.07    None        False               Low   
25             51   0.17    None        False               Low   
26             52   0.27    None        False               Low   
27             53   0.29    None        False               Low   
28             56   0.22    None        False               Low   
29             57   0.20    None        False               Low   
...           ...    ...     ...          ...               ...   
20974       27174   0.41    None        False               Low   
20975       27176   0.13    None        False               Low   
20976       27177   0.13    None        False               Low   
20977       27178   0.20    None        False               Low   
20978       27179   0.30    None        False               Low   
20979       27180   0.26    None         True              High   
20980       27181   0.17    None         True              High   
20981       27182   0.21    None         True              High   
20982       27183   0.25    None        False               Low   
20983       27184   0.38    None        False               Low   
20984       27185   0.22    None        False               Low   
20985       27186   0.51    None        False               Low   
20986       27187   0.49    None        False               Low   
20987       27189   0.15    None        False               Low   
20988       27190   0.40    None        False               Low   
20989       27191   0.55    None         True              High   
20990       27192   0.21    None        False               Low   
20991       27193   0.41    None         True              High   
20992       27194   0.23    None        False               Low   
20993       27197   0.18    None         True              High   
20994       27198   0.47    None         True              High   
20995       27199   0.22    None         True              High   
20996       27200   0.12    None         True              High   
20997       27201   0.49    None        False               Low   
20998       27202   0.12    None        False               Low   
20999       27203   0.30    None        False               Low   
21000       27204   0.65    None        False               Low   
21001       27205   0.19    None        False         

In [32]:
print(len(events_train["event_map_path"]))
print(len(events_train["event_map_path"].dropna()))
print(len(events_train["ground_map_path"]))
print(len(events_train["ground_map_path"].dropna()))
print(len(events_train["z_map_path"]))
print(len(events_train["z_map_path"].dropna()))

21004
21004
21004
21004
21004
21000


In [27]:
events_test = pd.read_csv("new_events_test.csv")

In [28]:
# events_train.to_csv("events_train.csv")

In [29]:
# events_test.to_csv("events_test.csv")

# Create Dataset

In [30]:
dataset_train = XChemDataset(events_train,
                             mode="RefMovev2",
                 grid_size=grid_size,
                 grid_step=grid_step,
                            network_type=network_type)

NameError: name 'XChemDataset' is not defined

In [ ]:
len(dataset_train)

In [ ]:
# for i, sample in enumerate(dataset_train):
#     x = sample["x"]
#     y = sample["y"]
#     print("{}: {}: {}".format(i, x.shape, y.shape))

In [ ]:
sample = dataset_train[np.random.randint(len(dataset_train))]

In [ ]:
x = sample["x"]

In [ ]:
sample["y"]

In [ ]:
# x.shape

In [ ]:
# x_2 = dataset_train[10]["x"]

In [ ]:
x_np_slice_2d_ref = x[0, :, :, int(x.shape[2]/2)]


In [ ]:
x_np_slice_2d_mov = x[1, :, :, int(x.shape[2]/2)]


In [ ]:
# x_2_np_slice_2d_2 = x_2[:, :, int(x.shape[2]/2)]


In [ ]:
# Plot 
plt.imshow(x_np_slice_2d_ref)

In [ ]:
plt.imshow(x_np_slice_2d_mov)

In [ ]:
x.std()

# Create Dataloader

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset_train,
                                         batch_size=16, 
                                         shuffle=True,
                                         num_workers=8)

# Define Model

In [ ]:
model = ClassifierV1(32,
                        grid_dimension=grid_size)

In [ ]:
print(model)

# Define loss function

In [ ]:
criterion = nn.MSELoss()
criterion = nn.BCELoss()

# Define optimizer

In [ ]:
optimizer = optim.Adam(model.parameters(), 
                       lr=0.0001)


# Fit Model

In [ ]:
num_epochs = 500

In [ ]:
running_loss = 0

In [ ]:
for epoch in range(num_epochs):
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        x = data["x"]
        y = data["y"]
        y = y.view(-1,2)
#         y = y + 1
#         print("meanx")
#         print(x.mean())
#         print(x.shape)
#         print(y.shape)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(x)
#         print(outputs)
#         print(outputs)
        loss = criterion(outputs, y)
#         print(loss)
        loss.backward()
        optimizer.step()
        
        # RECORD LOSS
        running_loss += loss.item()
#         print("Iteration number: {}, loss: {}".format(i, loss.item()))
#         print("{}".format([round(float(x), 3) for x in outputs]))
#         print("{}".format([round(float(x), 3) for x in y]))
#         print("{}".format([x for x in outputs]))
#         print("{}".format([x for x in y]))



        # print statistics per epoch
        if i % 30 == 29:    # print every 100 mini-batches
            print("Loss at epoch {}, iteration {} is {}".format(epoch,
                                                                 i,
                                                                 running_loss / i))
            print("{}".format([x for x in outputs]))
            print("{}".format([x for x in y]))
            print("#################################################")
    print("###################################")
#         print('[%d, %5d] loss: %.3f' %
#               (epoch + 1, i + 1, running_loss / 20))
    print("Loss for epoch {}: {}".format(epoch, running_loss))
    print(outputs)
    print(y)
    print(loss)
    running_loss = 0.0
            
    torch.save(model.state_dict(), "model_params_{}_{}_{}_{}_{}.pt".format(grid_size,
                                                             grid_step,
                                                                           network_type,
                                                                    network_version,
                                                                       dataset_version))

In [ ]:
torch.save(model.state_dict(), "model_params_{}_{}_{}_{}.pt".format(grid_size,
                                                             grid_step,
                                                                    network_version,
                                                                       dataset_version))

In [ ]:
model(x)

In [ ]:
y